In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob
import pickle as pkl
import re
import ast

In [2]:
def data_loader(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()

    # initializing variables
    n = p = m = noise_level = 0
    data = []
    for line in lines:
        try:
            if "Running trials for" in line:
                n = int(re.search(r'n = (\d+)', line).group(1))
                p = int(re.search(r'p = (\d+)', line).group(1))
                m = int(re.search(r'm = (\d+)', line).group(1))
            elif "Cross validating alpha under noise level:" in line:
                noise_level = float(re.search(r'noise level:  (\d+\.\d+)', line).group(1))
            elif "Trial:" in line:
                trial_num = int(re.search(r'Trial:  (\d+)', line).group(1))
                params = ast.literal_eval(re.search(r"Best params:  ({.*?})", line).group(1))
                error = float(re.search(r"Lowest Error:  (\d+\.\d+)", line).group(1))
                params["trial_num"] = trial_num
                params["lowest_error"] = error
                params["n"] = n
                params["p"] = p
                params["m"] = m
                params["noise_level"] = noise_level
                data.append(params)
        except Exception as e:
            print(f"Error parsing line: {e}")

    return pd.DataFrame(data)

In [48]:
# specify the directory you want to search in
directory = 'July 6th'

# use glob to match the file pattern '*.out'
files = glob.glob(os.path.join(directory, '*.out'))

In [49]:
# loop over the list of files from the glob.glob() method
final_results = pd.DataFrame()
for filename in files:
    df = data_loader(filename)
    final_results = final_results.append(df)

/var/folders/r9/cr3fkwq558n9mv4z9n1c01k00000gn/T/ipykernel_39193/3107640293.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_results = final_results.append(df)
/var/folders/r9/cr3fkwq558n9mv4z9n1c01k00000gn/T/ipykernel_39193/3107640293.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_results = final_results.append(df)


In [16]:
filename = "July 14/BOMP_600_1000_20_r_0714_0714-215912.pkl"
with open("July 14/BOMP_600_1000_20_r_0714_0714-215912.pkl", "rb") as file:
    omp_data = pkl.load(file)[0]

In [17]:
omp_data['log'][0]['best_bag_k_error_matrix']

array([[3.0000000e+02, 1.4000000e+02, 1.2343298e-03]])

In [44]:
filename = "July 15/Baseline_BOMP_600_1000_20_r_0715_0715-175014.pkl"
with open(filename, "rb") as file:
    omp_data = pkl.load(file)[0]

In [65]:
temp_log  = omp_data['log'][0]

In [46]:
omp_data['parameters']

{'n': 600,
 'p': 1000,
 'm': 20,
 'noise_level_lst': [0.02, 0.04, 0.06, 0.08, 0.1],
 'model_name': 'BOMP',
 'cv_num': 5,
 'trial_num': 10,
 'param_grid': {},
 'fixed_params': {'ignore_warning': True,
  'random_seed': 1,
  'select_atom_percent': 0,
  'K_lst': [50, 100, 150, 200, 250, 300, 350, 400]}}

In [5]:
# specify the directory you want to search in
directory = 'Results_Analysis/outputs'

# use glob to match the file pattern '*.out'
Baseline_Files = glob.glob(os.path.join(directory, 'Baseline*.pkl'))
Baseline_Files

In [13]:
omp_data['log'][0]

{'noise_level': 0.02,
 'trial': 0,
 'cv_error_lst': array([0.00394813]),
 'lowest_cv_error': 0.003948126449573877,
 'best_params': {'best_k': 50},
 'param_lst': [{'best_k': 50}],
 'testing_error': 0.0034058152626021427,
 'best_bag_k_error_series': array([0.1034963 , 0.10382416, 0.10448102, 0.10520692, 0.1052923 ,
        0.10550158, 0.10557526, 0.10563439])}

In [16]:
temp_Baseline_file = Baseline_Files[0]

with open(temp_Baseline_file, "rb") as file:
    omp_data = pkl.load(file)[0]
omp_df = pd.DataFrame()

for i in range(len(omp_data['log'])):
    temp_df = pd.DataFrame()
    temp_df['noise_level'] = [omp_data['log'][i]['noise_level']]

    temp_df['trial_num'] = [omp_data['log'][i]['trial']]

    temp_df['testing_error'] = [omp_data['log'][i]['testing_error']]

    temp_df['training_error'] = [omp_data['log'][i]['lowest_cv_error']]

    temp_df['best_k'] = [omp_data['log'][i]['best_params']['best_k']]
    omp_df = pd.concat([omp_df, temp_df])

omp_error = omp_df.copy()[['noise_level', 'testing_error', 'training_error']]
omp_error = omp_error.groupby('noise_level').mean().reset_index()
omp_error

,noise_level,testing_error,training_error
0,0.02,0.001691,0.002677
1,0.04,0.003227,0.004198
2,0.06,0.005743,0.006922
3,0.08,0.009327,0.010741
4,0.10,0.014189,0.015809


In [56]:
bomp_files = glob.glob(os.path.join(directory, 'BOMP*.pkl'))
bomp_files = ['Results_Analysis/outputs/BOMP_600_1000_20_nr_0715_0715-155336.pkl','Results_Analysis/outputs/BOMP_600_1000_20_r_0715_0715-155333.pkl']

True

In [70]:
len(bomp_data['log'])

29

In [74]:
bomp_df = pd.DataFrame()

for i in range(len(bomp_files)):
    temp_bomp_file = bomp_files[i]
    with open(temp_bomp_file, "rb") as file:
        bomp_data = pkl.load(file)[0]
    temp_df = pd.DataFrame()
    for k in range(len(bomp_data['log'])):
        temp_df = pd.DataFrame()
        temp_df['noise_level'] = [bomp_data['log'][k]['noise_level']]

        temp_df['trial_num'] = [bomp_data['log'][k]['trial']]

        temp_df['testing_error'] = [bomp_data['log'][k]['testing_error']]

        temp_df['training_error'] = [bomp_data['log'][k]['lowest_cv_error']]

        temp_df['best_k'] = [bomp_data['log'][k]['best_params']['best_k']]

        temp_df['best_bag'] = [bomp_data['log'][k]['best_params']['best_bag']]

        temp_df['replaced_flag'] = bomp_data['parameters']['fixed_params']['replace_flag']
        
        bomp_df = pd.concat([bomp_df, temp_df])
bomp_df

,noise_level,trial_num,testing_error,training_error,best_k,best_bag,replaced_flag
0,0.02,0,0.001727,0.002563,50,100,False
0,0.02,1,0.000440,0.007262,50,200,False
0,0.02,2,0.000450,0.000586,50,300,False
0,0.02,3,0.000634,0.000589,50,200,False
0,0.02,4,0.000609,0.000585,50,300,False
...,...,...,...,...,...,...,...
0,0.06,4,0.005298,0.007579,200,300,True
0,0.06,5,0.005206,0.007402,250,300,True
0,0.06,6,0.005906,0.007175,300,300,True
0,0.06,7,0.006005,0.009212,300,200,True


In [73]:
bomp_df

,noise_level,trial_num,testing_error,training_error,best_k,best_bag,replaced_flag
0,0.02,0,0.001727,0.002563,50,100,False
0,0.02,1,0.000440,0.007262,50,200,False
0,0.02,2,0.000450,0.000586,50,300,False
0,0.02,3,0.000634,0.000589,50,200,False
0,0.02,4,0.000609,0.000585,50,300,False
...,...,...,...,...,...,...,...
0,0.06,4,0.005298,0.007579,200,300,True
0,0.06,5,0.005206,0.007402,250,300,True
0,0.06,6,0.005906,0.007175,300,300,True
0,0.06,7,0.006005,0.009212,300,200,True


In [90]:
bomp_df

,noise_level,trial_num,testing_error,training_error,best_k,best_bag,replaced_flag
0,0.02,0,0.001727,0.002563,50,100,False
0,0.02,1,0.000440,0.007262,50,200,False
0,0.02,2,0.000450,0.000586,50,300,False
0,0.02,3,0.000634,0.000589,50,200,False
0,0.02,4,0.000609,0.000585,50,300,False
...,...,...,...,...,...,...,...
0,0.06,4,0.005298,0.007579,200,300,True
0,0.06,5,0.005206,0.007402,250,300,True
0,0.06,6,0.005906,0.007175,300,300,True
0,0.06,7,0.006005,0.009212,300,200,True


In [85]:
bomp_error = bomp_df.copy()[['noise_level', 'testing_error', 'training_error','replaced_flag']]
bomp_error = bomp_error.groupby(['noise_level','replaced_flag']).mean().reset_index()
bomp_error

,noise_level,replaced_flag,testing_error,training_error
0,0.02,False,0.000812,0.100660
1,0.02,True,0.000762,0.001850
2,0.04,False,0.002676,0.005583
3,0.04,True,0.002724,0.004413
4,0.06,False,0.005780,0.009437
5,0.06,True,0.005554,0.007699
6,0.08,False,0.009985,0.014892
7,0.10,False,0.016597,0.021413


In [86]:
with open(temp_bomp_file, "rb") as file:
    bomp_data = pkl.load(file)[0]

In [87]:
bomp_training_error = bomp_error.drop('testing_error', axis=1)[['noise_level','training_error']].groupby(['noise_level']).min().reset_index()
bomp_training_error

,noise_level,training_error
0,0.02,0.001850
1,0.04,0.004413
2,0.06,0.007699
3,0.08,0.014892
4,0.10,0.021413


In [89]:
bomp_error[bomp_error['training_error'].isin(bomp_training_error['training_error'])]

,noise_level,replaced_flag,testing_error,training_error
1,0.02,True,0.000762,0.001850
3,0.04,True,0.002724,0.004413
5,0.06,True,0.005554,0.007699
6,0.08,False,0.009985,0.014892
7,0.10,False,0.016597,0.021413
